In [141]:
import imageio.v2 as imageio
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug import parameters as iap
import glob
import numpy as np
import cv2
import random


In [ ]:
#import images
path = glob.glob("D:\DemoAugmentation\demo\*.jpg")
cv_img = []

for img in path:
    n = cv2.imread(img)
    width, height,_ = n.shape
    if width == 3456 and height == 3112:
        #r_n = n.resize(1728,1556,3)
        cv_img.append(n)
 
ia.imshow(np.hstack(cv_img)) 
#cv_img = cv_img*10
#img = imageio.imread() 

In [ ]:
print(len(cv_img))
print(type(cv_img))
print(np.shape(cv_img))

In [ ]:
#Augmentation Images  
images_aug = seq(images=cv_img)
print("Augmented:")
ia.imshow(np.hstack(images_aug))

In [102]:
#Sequential for Augemntation
ia.seed(1)

seq = iaa.Sequential([
    
    #Geometry
    iaa.Sometimes(0.25, 
            iaa.OneOf([
                iaa.Affine(rotate=(-25, 25)),
                iaa.Fliplr(0.2), #horizontally 
                iaa.Flipud(0.2), #vertically 

                iaa.AddElementwise((1,30)), 
                iaa.JpegCompression(compression=(1,10)), 
            ]),
        ),
    
    # #Color Space 
    iaa.Sometimes(0.15, 
            iaa.OneOf([    
                iaa.RemoveSaturation(0.2),
                iaa.AddToSaturation((-15,15)),
                iaa.AddToHueAndSaturation((-10,10), per_channel=True),
                iaa.AddToHue((-15,15)),
                #iaa.ChangeColorTemperature((5300,7200)), #Error if batch >= 2
            ]),
        ),
    
    #Noise
    iaa.Sometimes(0.25, 
            iaa.OneOf([
                iaa.AdditiveGaussianNoise(scale=(1,7),per_channel=False), #Noise
                iaa.AdditiveGaussianNoise(scale=(1,10),per_channel=True), #Color Noise

                iaa.AdditiveLaplaceNoise(scale=(1,5),per_channel=False),
                iaa.AdditiveLaplaceNoise(scale=(1,5),per_channel=True),

                iaa.AdditivePoissonNoise((1,5),per_channel=False), 
                iaa.AdditiveLaplaceNoise(scale=(1,5),per_channel=True),
            ]),
        ),

    #Contrast + Brightness
    iaa.Sometimes(0.35, 
            iaa.OneOf([    

                iaa.AddToBrightness((-20, 20)),
                iaa.MultiplyBrightness((0.85,1.15)),
                iaa.MultiplyAndAddToBrightness(mul=(0.85, 1.15), add=(-20, 20)),

                iaa.GammaContrast((1.0 , 2.0)), 
                iaa.LogContrast(gain=(0.9,1.1)),
                iaa.LinearContrast((0.9,1.2)),
            ]),
        ),
    
    #Blur
    iaa.Sometimes(0.25, 
            iaa.OneOf([    
                iaa.GaussianBlur(sigma=(0.0,1.0)),
                iaa.MedianBlur(k=(1,3)), 
                iaa.AverageBlur(k=((1,3), (1,3))),
                iaa.MotionBlur(k=5, angle=[-25, 25]),
                iaa.BilateralBlur(d=(1,5), sigma_color=(10, 250), sigma_space=(10, 250)),
            ]),
        ),
])


In [136]:
#import images
#path = glob.glob("D:\DemoAugmentation\_0\*.jpg")
#path = glob.glob("D:\DemoAugmentation\_1\*.jpg")
path = glob.glob("D:\DemoAugmentation\_2\*.jpg")
#path = glob.glob("D:\DemoAugmentation\_3\*.jpg")

# save_path0 = "D:\DemoAugmentation\Aug0"
# save_path1 = "D:\DemoAugmentation\Aug1"
# save_path2 = "D:\DemoAugmentation\Aug2"
# save_path3 = "D:\DemoAugmentation\Aug3"

# split_path0 = "D:\DemoAugmentation\demo0" 
# split_path1 = "D:\DemoAugmentation\demo1" 
# split_path2 = "D:\DemoAugmentation\demo2" 
# split_path3 = "D:\DemoAugmentation\demo3" 

total_files = 0 #total images in the files
others_img = 0 #unqualified name count
ip_img = [] #array for load image in
pter = 1 #pointer for batch proccess
j = 1 #image name


In [ ]:
for img in path:
        total_files += 1
print ("Total files is: ",total_files)
total_files  = 0
#total_files = 382
#total_files = 447
#total_files = 105
#total_files = 12

In [ ]:
#Count files in directory 
for img in path:
    n = cv2.imread(img)
    width, height,_ = n.shape
    if width == 3456 and height == 3112:
        total_files += 1
print ("Total files is: ",total_files)
#total_files = 0
# level 0 = 382 same size images
# level 1 = 447 same size images
# level 2 = 105 same size images
# level 3 = 12 same size images

In [ ]:
#------------------------------IMAGE-AUGMENTATION-------------------------------------
SCALE = 20.5655527 # percent of original size
AUGTIME = 10 #augment time
BATCH = 10 #Batch size
j = 800 

for a in range(AUGTIME):
    print("Augment number: ",a)
    others_img = 0
    pter = 0

    for img in path:
        n = cv2.imread(img)
        width, height,_ = n.shape
        
        # Only get image with same dimensions
        if width == 3456 and height == 3112:
            RGB_img = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
            print(RGB_img.shape)
    
            #Crop to 1:1 ratio
            cropped_img = RGB_img[172:3284, 0:3112]
            print(cropped_img.shape)
            #resize Image
            print("Resized to: ")
            w = int(cropped_img.shape[1] * SCALE / 100)
            h = int(cropped_img.shape[0] * SCALE / 100)
            dim = (w, h)
            resized_img = cv2.resize(cropped_img, dim, interpolation = cv2.INTER_AREA)
            print(resized_img.shape)
    
            #append to the list
            ip_img.append(resized_img)
            #ip_img.extend(resized_img)
            #ip_img += resized_img
            pter += 1
            print(len(ip_img))
    
            if pter % BATCH == 0 and pter != 0:
                #ip_img = ip_img*AUGTIME
                augmented = seq(images=ip_img)

                #save image augmented into the file 
                for i,augment in enumerate(augmented):
                    cv2.imwrite("D:\DemoAugmentation\Aug2\%i.jpg" % (j,), cv2.cvtColor(augment, cv2.COLOR_RGB2BGR)) 
                    j+=1 
                    #j=random.randint(1,5)

                print("Image saved")
                print(pter)
                print("---------------------------------------------------------------")
                ip_img = []
                
                
            elif (total_files - pter < BATCH): 
                #ip_img = ip_img*AUGTIME
                augmented_left = seq(images=ip_img)
                
                #save image augmented into the file 
                for i,augment_img in enumerate(augmented_left):
                    cv2.imwrite("D:\DemoAugmentation\Aug2\%i.jpg" % (j,), cv2.cvtColor(augment_img, cv2.COLOR_RGB2BGR))
                    j+=1 
                    #j=random.randint(1,5)

                print("Image saved")
                print(pter)
                print("---------------------------END_OF_LAST_BATCH------------------------------------")
                ip_img = []
                
        else:
            #Save other dimensions image into another file
            RGB_img0 = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
            cv2.imwrite("D:\DemoAugmentation\demo2\%i.jpg" % (others_img,), cv2.cvtColor(RGB_img0, cv2.COLOR_RGB2BGR))
            others_img += 1  

print("Augmentaion Completed!")
print("Other size images: %i" % (others_img))

In [ ]:
#--------------------------Single-Image-Handling-----------------------------------------
path0 = glob.glob("D:\DemoAugmentation\demo2\*.jpg")
others_img = 0
O_img = [] 
j = 100

AUGTIME = 10
for _ in range (AUGTIME):
    for img in path0:

        O_img = [] 
        n = cv2.imread(img)
        RGB_img = cv2.cvtColor(n, cv2.COLOR_BGR2RGB)
        print(RGB_img.shape)
        width, height,_ = RGB_img.shape

        #Check ratio 1:1
        if width != height:
            if width > height :
                delta_width = abs(width-height)
                delta_height = 0

                pad_width = delta_width // 2
                pad_height = delta_height // 2

                RGB_img = cv2.copyMakeBorder(RGB_img, pad_height, pad_height, pad_width, pad_width, cv2.BORDER_CONSTANT, value = 0)

            elif width < height:
                delta_width = 0
                delta_height = abs(width-height)

                pad_width = delta_width // 2
                pad_height = delta_height // 2

                RGB_img = cv2.copyMakeBorder(RGB_img, pad_height, pad_height, pad_width, pad_width, cv2.BORDER_CONSTANT, value = 0)

        print(RGB_img.shape)
        width, height,_ = RGB_img.shape
        #Check ratio 1:1

        print("Resized to: (640,640)")
        w = 640
        h = 640
        dim = (w, h)
        if width >= 640: #Resize Big -> 640
            resized_img = cv2.resize(RGB_img, dim, interpolation = cv2.INTER_AREA)

        else: #Resize Small -> 640
            resized_img = cv2.resize(RGB_img, dim, interpolation = cv2.INTER_LINEAR)

        #append to the list
        O_img.append(resized_img)
        
        augmented = seq(images=O_img)

        #save image augmented into the file 
        for i,augment in enumerate(augmented):
            cv2.imwrite("D:\DemoAugmentation\Aug2\%i.jpg" % (j,), cv2.cvtColor(augment, cv2.COLOR_RGB2BGR)) 
            j+= random.randint(1,6)
            print("Image saved")
            print("---------------------------------------------------------------") 

    print("Augmentaion Completed!")
    print("Other size images: %i" % (others_img))